In [1]:
import matplotlib.pyplot as plt
from modeling.evaluate import Evaluate
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.preprocessing import MinMaxScaler
import warnings
import pandas as pd
import numpy as np
warnings.simplefilter(action="ignore")
plt.style.use("seaborn-v0_8-darkgrid")
pd.set_option('display.max_columns', 50)


In [2]:
df = pd.read_csv('database/DIM_PLAYER_GAMES.csv')
df_dfns = pd.read_csv('database/DIM_DEFENSE.csv').drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

In [3]:
df.columns

Index(['GAME_ORDER', 'YEAR', 'DATE', 'GAME_NUMBER', 'WEEK', 'PLAYER', 'AGE',
       'TEAM', 'OPPONENT', 'RESULT', 'RUSHING_ATT', 'RUSHING_YDS',
       'RUSHING_TD', 'RUSHING_1D', 'RUSHING_YBC', 'RUSHING_YBC_ATT',
       'RUSHING_YAC', 'RUSHING_YAC_ATT', 'RUSHING_BRKTKL', 'RUSHING_ATT_BR',
       'RECEIVING_TGT', 'RECEIVING_REC', 'RECEIVING_YDS', 'RECEIVING_TD',
       'RECEIVING_1D', 'RECEIVING_YBC', 'RECEIVING_YBC_R', 'RECEIVING_YAC',
       'RECEIVING_YAC_R', 'RECEIVING_ADOT', 'RECEIVING_BRKTKL',
       'RECEIVING_REC_BR', 'RECEIVING_DROP', 'RECEIVING_DROP_PCT',
       'RECEIVING_INT', 'RECEIVING_RAT', 'PASSING_CMP', 'PASSING_ATT',
       'PASSING_CMP_PCT', 'PASSING_YDS', 'PASSING_TD', 'PASSING_INT',
       'PASSING_RATE', 'PASSING_SK', 'PASSING_YDS1', 'PASSING_Y_A',
       'PASSING_AY_A', 'PPR'],
      dtype='object')

In [4]:
df_dfns.columns

Index(['Unnamed: 0.2', 'DEF_TEAM', 'OPP_CODE', 'YEAR', 'WEEK', 'SCORE_OPP',
       'PASSING_CMP', 'PASSING_ATT', 'PASSING_YDS', 'PASSING_TD',
       'PASSING_INT', 'PASSING_SK', 'PASSING_Y_A', 'PASSING_NY_A',
       'PASSING_CMP_PCT', 'PASSING_RATE', 'RUSHING_ATT', 'RUSHING_YDS',
       'RUSHING_Y_A', 'RUSHING_TD'],
      dtype='object')

In [5]:
df_dfns.sort_values(["DEF_TEAM", "YEAR", "WEEK"], inplace=True)

rolling_series = (
    df_dfns
    .groupby(["DEF_TEAM", "YEAR"])[["PASSING_YDS", "PASSING_TD", "PASSING_CMP", "RUSHING_YDS", "RUSHING_TD"]]
    .rolling(window=17, min_periods=1)
    .mean()
    .shift(periods=1)
)


In [6]:
df_dfns_rl = pd.DataFrame(rolling_series).reset_index()
df_dfns_rl['WEEK'] = df_dfns['WEEK']
df_dfns_rl['OPP_CODE'] = df_dfns['OPP_CODE']

In [7]:
# df_dfns.sort_values(["DEF_TEAM", "YEAR", "WEEK"], inplace=True)
# for i in interested:
#     # Calculate the rolling mean
#     rolling_series = (
#         df_dfns
#         .groupby(["DEF_TEAM", "YEAR"])[i]
#         .rolling(min_periods=1, window=17)
#         .mean()
#         .shift(periods=1)
#     )

#     # Reset the index to align with the original dataframe
#     rolling_series = rolling_series.reset_index(level=0, drop=True)

#     # scaler = MinMaxScaler()|
#     # scaled_values = scaler.fit_transform(rolling_series.values.reshape(-1, 1)).flatten()

#     # Assign the values to the new column
#     df_dfns[f"{i}_rolling"] = rolling_series

In [8]:
# df_dfns_j = df_dfns[
#     [
#         "YEAR",
#         "WEEK",
#         "DEF_TEAM",
#         "OPP_CODE",
#         "SCORE_OPP_rolling",
#         "PASSING_CMP_rolling",
#         "PASSING_ATT_rolling",
#         "PASSING_YDS_rolling",
#         "PASSING_TD_rolling",
#         "PASSING_INT_rolling",
#         "PASSING_SK_rolling",
#         "PASSING_Y_A_rolling",
#         "PASSING_NY_A_rolling",
#         "PASSING_CMP_PCT_rolling",
#         "PASSING_RATE_rolling",
#         "RUSHING_ATT_rolling",
#         "RUSHING_YDS_rolling",
#         "RUSHING_Y_A_rolling",
#         "RUSHING_TD_rolling",
#     ]
# ]

In [9]:
df_comb = pd.merge(df_dfns_rl, df, how='left', left_on=['YEAR', 'WEEK', 'OPP_CODE'], right_on=['YEAR', 'WEEK', 'TEAM']).sort_values(by=['PLAYER', 'DATE'])

In [10]:
df_comb['DEF_REC_POINTS'] = df_comb['PASSING_YDS_x']/10 + df_comb['PASSING_TD_x']*6 +  df_comb['PASSING_CMP_x']
df_comb['DEF_RB_POINTS'] =  df_comb['RUSHING_YDS_x']/10 + df_comb['RUSHING_TD_x']*6

In [11]:
df_comb['DATE'] = pd.to_datetime(df_comb['DATE'])

In [126]:

top_180_eval = [
    Evaluate(
        df=df_comb,
        player=name,
        steps=17,
        target='PPR',
        regressor = xgb.XGBRFRegressor(random_state=123),
        lags=9,
        exogs=[
         'DEF_REC_POINTS', 'DEF_RB_POINTS'],
    ).eval()
    for name in list(set(df_comb["PLAYER"].dropna()))
]

df_top_180 = pd.concat(top_180_eval)

Not enough data. Adjusting lags for De'Von Achane.
Not enough data. Adjusting steps for De'Von Achane.
Not enough data. Adjusting lags for Zach Charbonnet.
Not enough data. Adjusting steps for Zach Charbonnet.
Not enough data. Adjusting steps for Cade Otton.
Not enough data. Adjusting steps for Dontayvion Wicks.
Not enough data. Adjusting lags for Easton Stick.
Not enough data. Adjusting steps for Easton Stick.
Not enough data. Adjusting steps for Kyren Williams.
Not enough data. Adjusting steps for Zach Wilson.
Not enough data. Adjusting steps for Jayden Reed.
Not enough data. Adjusting steps for Josh Downs.
Not enough data. Adjusting steps for Rico Dowdle.
Not enough data. Adjusting lags for Bailey Zappe.
Not enough data. Adjusting steps for Bailey Zappe.
Not enough data. Adjusting steps for Nick Mullens.
Not enough data. Adjusting lags for Jahmyr Gibbs.
Not enough data. Adjusting steps for Jahmyr Gibbs.
Not enough data. Adjusting steps for Isaiah Likely.
Not enough data. Adjusting s

In [127]:
# df_top_181 = df_top_180.loc[(df_top_180['ACTUAL SEASON TOTAL']>120)&(df_top_180['LAGS']>=10)]
df_top_181 = df_top_180.loc[(df_top_180['ACTUAL SEASON TOTAL']>0)]

In [128]:
df_top_181.sort_values(by='SEASON MAPE').head(25)

,PLAYER,ACTUAL SEASON TOTAL,PREDICTED,SEASON MAPE,GAME MAPE,GAME RSME,GAMES PREDICTED,LAGS,REGRESSOR
0,Roschon Johnson,21.60,21.595816,0.000194,3.002848e+00,5.786080,5,5,"XGBRFRegressor(base_score=None, booster=None, ..."
0,Kenneth Walker III,186.10,186.300779,0.001079,4.434761e-01,3.920856,15,9,"XGBRFRegressor(base_score=None, booster=None, ..."
0,Josh Allen,429.36,428.180140,0.002748,1.707365e-01,5.448443,17,9,"XGBRFRegressor(base_score=None, booster=None, ..."
0,Davante Adams,323.10,321.931501,0.003617,1.911727e+00,14.425468,17,9,"XGBRFRegressor(base_score=None, booster=None, ..."
0,Javonte Williams,189.20,188.007561,0.006303,8.439520e-01,7.033493,17,9,"XGBRFRegressor(base_score=None, booster=None, ..."
0,Terry McLaurin,214.50,213.008297,0.006954,3.776670e+15,5.824635,17,9,"XGBRFRegressor(base_score=None, booster=None, ..."
0,Jonathan Taylor,242.10,243.852401,0.007238,4.949342e-01,5.925354,17,9,"XGBRFRegressor(base_score=None, booster=None, ..."
0,Adam Thielen,204.40,206.433631,0.009949,1.528576e+00,10.244318,17,9,"XGBRFRegressor(base_score=None, booster=None, ..."
0,Amari Cooper,295.10,289.579242,0.018708,7.073178e-01,11.755889,17,9,"XGBRFRegressor(base_score=None, booster=None, ..."
0,T.J. Hockenson,247.50,242.740310,0.019231,3.145586e-01,6.068754,17,9,"XGBRFRegressor(base_score=None, booster=None, ..."


In [129]:
np.mean(df_top_181['SEASON MAPE']), np.mean(df_top_181['GAME RSME']), np.median(df_top_181['GAME MAPE']), np.mean(df_top_181['GAME MAPE'])

(0.395109409390938, 8.57861920723517, 0.8448576907228684, 1277037595423506.0)

In [130]:
# np.sqrt(mean_squared_error(df_top_180['ACTUAL SEASON TOTAL'], df_top_180['PREDICTED']))

In [131]:
# np.sqrt(mean_squared_error(df_top_180['ACTUAL SEASON TOTAL'], df_top_180['PREDICTED']))

In [132]:
# top_180_pred = [
# Predict(
#     df=df, player=i, steps=17, regressor=GradientBoostingRegressor, lags=9
# ).predict_season() for i in list(set(df['PLAYER']))
# ]

# clean_data = [row for row in top_180_pred if row is not None]

In [133]:
# df_preds = pd.DataFrame(clean_data, columns=['PLAYER', 'PROJECTED POINTS'])

In [134]:
# df_2023 = df.loc[df['YEAR']==2023]
# df_players_2023 = df_2023.groupby(['PLAYER', 'TEAM'])['PPR'].sum().reset_index()
# df_players_2023.rename(columns={'PPR':'PPR_2023'}, inplace=True)

In [135]:
# df_adp = pd.read_csv('ADP_2024.csv')
# df_adp['Position'] = df_adp['Consensus Pos'].str[:2]
# df_adp['Target Round'] = (df_adp['Consensus'] // 10).astype(int) + 1

In [136]:
# df_merge1 = pd.merge(df_players_2023, df_preds,  how='inner', on='PLAYER')

In [137]:
# df_final = pd.merge(df_merge1, df_adp[['Player', 'Position', 'Consensus', 'Target Round']], left_on='PLAYER', right_on='Player', how='inner').drop(columns=['Player'])

In [138]:
# ranked_values = {
#     'QB': df_final[df_final['Position'] == 'QB'].nlargest(10, 'PROJECTED POINTS')['PROJECTED POINTS'].iloc[-1] if len(df_final[df_final['Position'] == 'QB']) >= 10 else df_final[df_final['Position'] == 'QB']['PROJECTED POINTS'].min(),
#     'TE': df_final[df_final['Position'] == 'TE'].nlargest(10, 'PROJECTED POINTS')['PROJECTED POINTS'].iloc[-1] if len(df_final[df_final['Position'] == 'TE']) >= 10 else df_final[df_final['Position'] == 'TE']['PROJECTED POINTS'].min(),
#     'WR': df_final[df_final['Position'] == 'WR'].nlargest(30, 'PROJECTED POINTS')['PROJECTED POINTS'].iloc[-1] if len(df_final[df_final['Position'] == 'WR']) >= 30 else df_final[df_final['Position'] == 'WR']['PROJECTED POINTS'].min(),
#     'RB': df_final[df_final['Position'] == 'RB'].nlargest(20, 'PROJECTED POINTS')['PROJECTED POINTS'].iloc[-1] if len(df_final[df_final['Position'] == 'RB']) >= 20 else df_final[df_final['Position'] == 'RB']['PPROJECTED POINTS'].min()
# }

# # Print the ranked PPR values for debugging
# print("Ranked PPR values:")
# print(ranked_values)

In [139]:
# def VBD(row):
#     category = row['Position']
#     ranked_ppr = ranked_values.get(category, 0) 
#     return row['PROJECTED POINTS'] - ranked_ppr

# # Apply the function to each row
# df_final['VBD'] = df_final.apply(VBD, axis=1)


In [140]:
# df_final['My Ranking'] = df_final['VBD'].rank(ascending=False, method='min').astype(int)
# df_final.sort_values(by='VBD', ascending=False).head(25)

In [141]:
# df_final.to_csv('Draft Strategy 2024.csv')